In [7]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
#input names of csv files created from web scrape
csv_name = 'imdb_2018.csv'
csv2_name = 'imdb_2019.csv'
#input name of json to create
json_name = 'imdb.json'

In [9]:
#import csv 1 to data frame
df1 = pd.read_csv(csv_name, index_col='Unnamed: 0')

#import csv 2 to data frame
df2 = pd.read_csv(csv2_name, index_col='Unnamed: 0')

#concatonate 2 data frames
df = pd.concat([df1, df2],ignore_index=True)
df.info()
df.describe()
df.shape
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16258 entries, 0 to 16257
Data columns (total 10 columns):
certificate    1542 non-null object
genre          15712 non-null object
gross          446 non-null float64
id             16258 non-null object
imdb_rating    6607 non-null float64
metascore      653 non-null float64
runtime        9259 non-null object
title          16258 non-null object
votes          6611 non-null float64
year           16153 non-null object
dtypes: float64(4), object(6)
memory usage: 1.2+ MB


,certificate,genre,gross,id,imdb_rating,metascore,runtime,title,votes,year
0,PG,"['Animation', 'Action', 'Adventure']",189936471.0,tt4633694,8.5,87.0,117 min,Spider-Man: Into the Spider-Verse,160975.0,['2018']
1,PG-13,"['Action', 'Adventure', 'Fantasy']",334826690.0,tt1477834,7.2,55.0,143 min,Aquaman,211622.0,['2018']
2,R,"['Crime', 'Drama', 'Thriller']",103804407.0,tt7959026,7.1,58.0,116 min,The Mule,32910.0,['2018']
3,PG-13,"['Biography', 'Comedy', 'Drama']",84068561.0,tt6966692,8.3,69.0,130 min,Green Book,158958.0,['2018']
4,PG-13,"['Biography', 'Drama', 'Music']",215983200.0,tt1727824,8.1,49.0,134 min,Bohemian Rhapsody,311167.0,['2018']


In [10]:
all_genres = [x.strip('[]').strip("''").replace("', '"," ").split(" ") if type(x)==str else 'NaN' for x in df['genre']]

unique_genres = []
for item in all_genres:
    unique_genres.extend(item)
    
unique_genres = list(set(unique_genres))
unique_genres.remove('N')
unique_genres.remove('a')

In [11]:
unique_genres

['War',
 'Western',
 'Animation',
 'Thriller',
 'Adventure',
 'Sci-Fi',
 'Biography',
 'Sport',
 'Reality-TV',
 'News',
 'Mystery',
 'Music',
 'Action',
 'Talk-Show',
 'Musical',
 'Horror',
 'Fantasy',
 'Documentary',
 'Crime',
 'Short',
 'Drama',
 'Family',
 'History',
 'Comedy',
 'Romance']

In [12]:
df_genres = pd.DataFrame(unique_genres, columns=['genre'])
df_genres

,genre
0,War
1,Western
2,Animation
3,Thriller
4,Adventure
5,Sci-Fi
6,Biography
7,Sport
8,Reality-TV
9,News


In [13]:
import json
json_name = 'genres.json'
df_genres.to_json(json_name, 'records')

In [14]:
with open(json_name) as json_file:  
    genre_data = json.load(json_file)

In [15]:
create_query = """CREATE TABLE IF NOT EXISTS genres
(id INTEGER PRIMARY KEY, genre TEXT)"""

In [16]:
import sqlite3
conn = sqlite3.connect('movies.db')
c = conn.cursor()

def create_table(query):
    c.execute(query)

def close_c_conn():
    c.close()
    conn.close()

In [17]:
c.execute('''DROP TABLE genres''')
create_table(create_query)

In [18]:
def genre_entry(genre_data):
 
    for i in range(len(genre_data)):
        genre = genre_data[i]['genre']
        
        c.execute("INSERT INTO genres (id, genre) VALUES (?,?)",
          (i, genre))
        
    conn.commit()

In [19]:
genre_entry(genre_data)

In [20]:
pd.read_sql_query("select * from genres;", conn)

,id,genre
0,0,War
1,1,Western
2,2,Animation
3,3,Thriller
4,4,Adventure
5,5,Sci-Fi
6,6,Biography
7,7,Sport
8,8,Reality-TV
9,9,News
